# Align to combined macaque and canine reference

In [2]:
bp = "PRJNA665227"
fastq_folder = f"/home/laura/projects/virus-watch-data/{bp}/raw"
sample_batch_file = f"{fastq_folder}/batch.txt"

In [11]:
# Ensembl version 110
canine_cdna = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.cdna.all.fa.gz"
canine_cdna_amb = "/home/laura/projects/virus-watch-data/canine_ref/ambigious_Canis_lupus_familiaris.ROS_Cfam_1.0.cdna.all.fa.gz"
macaque_cdna = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.cdna.all.fa.gz"
macaque_cdna_amb = "/home/laura/projects/virus-watch-data/rhesus_ref/ambigious_Macaca_mulatta.Mmul_10.cdna.all.fa.gz"
# Ensembl version 109
canine_dna = "/home/laura/projects/virus-watch-data/canine_ref/Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz"
canine_dna_amb = "/home/laura/projects/virus-watch-data/canine_ref/ambigious_Canis_lupus_familiaris.ROS_Cfam_1.0.dna.toplevel.fa.gz"
macaque_dna = "/home/laura/projects/virus-watch-data/rhesus_ref/Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz"
macaque_dna_amb = "/home/laura/projects/virus-watch-data/rhesus_ref/ambigious_Macaca_mulatta.Mmul_10.dna.toplevel.fa.gz"

canine_macaque_fasta = "/home/laura/projects/virus-watch-data/canine_macaque_ref/combined.cdna_dna_ambigious.fa.gz"

kallisto = "/home/laura/projects/kallisto/build/src/kallisto"
bustools = "/home/laura/projects/bustools/build/src/bustools"

### Align to PalmDB with new dlist implementation that also masks ambigious kmers (kallisto in branch new2)

Create dlist files:

In [12]:
%%time
# Replace all headers from ">string" to ">>" to tell kallisto we want to extract ambigious kmers from these
!gzip -dc $canine_cdna | sed '/^>/ s/.*/>>/' | gzip -c > $canine_cdna_amb
!gzip -dc $macaque_cdna | sed '/^>/ s/.*/>>/' | gzip -c > $macaque_cdna_amb
!gzip -dc $canine_dna | sed '/^>/ s/.*/>>/' | gzip -c > $canine_dna_amb
!gzip -dc $macaque_dna | sed '/^>/ s/.*/>>/' | gzip -c > $macaque_dna_amb

CPU times: user 21.4 s, sys: 4.82 s, total: 26.2 s
Wall time: 19min 59s


In [13]:
%%time
# Concatenate normal + ambigious cdna and dna from macaque and dog
!cat $canine_cdna $macaque_cdna $canine_dna $macaque_dna $canine_cdna_amb $macaque_cdna_amb $canine_dna_amb $macaque_dna_amb > $canine_macaque_fasta

CPU times: user 57.9 ms, sys: 14.2 ms, total: 72.1 ms
Wall time: 3.24 s


Generate virus index:

In [14]:
virus_fasta = "/home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa"
virus_index = "/home/laura/projects/virus-watch-data/virus_ref/kallisto_aa/noduplicates_rhesus_canine_cdna_dna_ambigious.idx"
virus_t2g = "/home/laura/projects/virus-watch-data/virus_ref/nodup_clu_t2g.txt"

In [23]:
# Generate virus reference index (with default d-list-overhang = 3)
!/usr/bin/time -v $kallisto index \
    -t 30 \
    --aa \
    --d-list=$canine_macaque_fasta \
    -i $virus_index \
    $virus_fasta


[index] --d-list-overhang was set to 3 (with --aa, the d-list overhang must be >= 3)
[build] loading fasta file /home/laura/projects/virus-watch-data/virus_ref/uniques_noduplicates.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508840 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::cons

Align to PalmDB and correct barcodes using host onlist:

In [24]:
out_folder = f"{fastq_folder}/../full_index_dna_cdna_amb/virus"

In [25]:
sample_batch_file

'/home/laura/projects/virus-watch-data/PRJNA665227/raw/batch.txt'

In [26]:
%%time
!/usr/bin/time -v $kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t 30 \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0


[bus] will try running read files supplied in batch file
[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,757
[index] number of D-list k-mers: 90,815
[quant] running in single-end mode
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698516_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698524_2.fastq.gz
[quant] will process file 1: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_1.fastq.gz
[quant] will process file 2: /home/laura/projects/virus-watch-data/PRJNA665227/raw/SRR12698536_2.fastq.gz
[quant] will process file 1: /home/la

In [27]:
host_out_folder = f"/home/laura/projects/virus-watch-data/{bp}/full_index_stringent_dlist_flank1/host_canine"

In [28]:
%%time
!$bustools sort \
    -m 4G \
    -t 40 \
    -o $out_folder/output_sorted.bus \
    $out_folder/output.bus

!$bustools correct \
    -w $host_out_folder/bustools_whitelist.txt \
    -o $out_folder/output_sorted_corrected.bus \
    $out_folder/output_sorted.bus

!$bustools sort \
    -m 4G \
    -t 40 \
    -o $out_folder/output_sorted_corrected_sorted.bus \
    $out_folder/output_sorted_corrected.bus

!$bustools count \
    --genecounts \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted_corrected_sorted.bus

partition time: 4s
partition time: 2.84s
Read in 236602929 BUS records
reading time 6.48s
sorting time 66.39s
writing time 0s
Found 775516 barcodes in the on-list
Processed 66670658 BUS records
In on-list = 13884218
Corrected    = 15683555
Uncorrected  = 37102885
partition time: 0.78s
 all fits in buffer
Read in 29567773 BUS records
reading time 0.6s
sorting time 6.52s
writing time 1.05s
CPU times: user 3.57 s, sys: 767 ms, total: 4.34 s
Wall time: 2min 58s


Sort and count without barcode correction to retain all cells:

In [29]:
out_folder_uncorrected = f"{fastq_folder}/../full_index_dna_cdna_amb/virus_uncorrected"

In [30]:
!$bustools sort \
    -m 4G \
    -t 40 \
    -o $out_folder_uncorrected/output_sorted.bus \
    $out_folder/output.bus

!$bustools count \
    --genecounts \
    -o $out_folder_uncorrected/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder_uncorrected/output_sorted.bus

partition time: 4.49s
partition time: 2.67s
Read in 236602929 BUS records
reading time 3.48s
sorting time 69.04s
writing time 0s
